In [ ]:
import pandas as pd
from IPython.display import HTML, display
from ipywidgets import interact

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

cols_to_print = ["number", "title", "repository"]

In [ ]:
df = pd.read_csv("github-activity.csv")

cond = [column.startswith("Unnamed:") for column in df.columns]
for column in df.columns[cond].tolist():
    df.drop(column, axis=1, inplace=True)

month_cond = ["last_month" in filter_name for filter_name in df["filter"]]
week_cond = ["last_week" in filter_name for filter_name in df["filter"]]

monthly_df = df[month_cond]
monthly_df = monthly_df[monthly_df["state"] == "closed"]
weekly_df = df[week_cond]

# 🚀 Activity Summary

## 📮 Closed in the Last Month

In [ ]:
repo_names = ["All"] + sorted(monthly_df["repo_name"].unique().tolist())
sort_columns = ["closed_at", "repo_name"]


@interact
def view_monthly_summmary(
    repo=repo_names,
    sort=sort_columns,
    sort_ascending=False,
    assigned=False,
    created=False,
    issues=False,
    pull_requests=False,
):
    if sort != "repo_name":
        columns_to_print = cols_to_print + [sort]
    else:
        columns_to_print = cols_to_print

    if repo == "All":
        filtered = monthly_df.sort_values(sort, ascending=sort_ascending)
    else:
        filtered = monthly_df.loc[monthly_df["repo_name"] == repo].sort_values(
            sort, ascending=sort_ascending
        )

    if assigned and (not created):
        # Only show tickets that I was assigned to
        cond = [
            filter_name.startswith("assigned") for filter_name in monthly_df["filter"]
        ]
        filtered = filtered[cond]
    elif (not assigned) and created:
        # Only show tickets that I created
        cond = [
            filter_name.startswith("created") for filter_name in monthly_df["filter"]
        ]
        filtered = filtered[cond]
    else:
        # No change. Show everything.
        pass

    if issues and (not pull_requests):
        filtered = filtered[filtered["pull_request"] == False]
    elif (not issues) and pull_requests:
        filtered = filtered[filtered["pull_request"] == True]
    else:
        # No change. Show everything.
        pass

    print(f"Total: {len(filtered[columns_to_print])}")
    display(HTML(filtered[columns_to_print].to_html(escape=False)))

## ✅ Activity in the Last Week

In [ ]:
repo_names = ["All"] + sorted(weekly_df["repo_name"].unique().tolist())
sort_columns = ["created_at", "updated_at", "repo_name"]


@interact
def view_weekly_summmary(
    repo=repo_names,
    sort=sort_columns,
    sort_ascending=False,
    assigned=False,
    created=False,
    issues=False,
    pull_requests=False,
):
    if sort != "repo_name":
        columns_to_print = cols_to_print + [sort]
    else:
        columns_to_print = cols_to_print

    if repo == "All":
        filtered = weekly_df.sort_values(sort, ascending=sort_ascending)
    else:
        filtered = weekly_df.loc[weekly_df["repo_name"] == repo].sort_values(
            sort, ascending=sort_ascending
        )

    if assigned and (not created):
        # Only show tickets that I was assigned to
        cond = [
            filter_name.startswith("assigned") for filter_name in weekly_df["filter"]
        ]
        filtered = filtered[cond]
    elif (not assigned) and created:
        # Only show tickets that I created
        cond = [
            filter_name.startswith("created") for filter_name in weekly_df["filter"]
        ]
        filtered = filtered[cond]
    else:
        # No change. Show everything.
        pass

    if issues and (not pull_requests):
        filtered = filtered[filtered["pull_request"] == False]
    elif (not issues) and pull_requests:
        filtered = filtered[filtered["pull_request"] == True]
    else:
        # No change. Show everything.
        pass

    filtered.dropna(inplace=True)

    print(f"Total: {len(filtered[columns_to_print])}")
    display(HTML(filtered[columns_to_print].to_html(escape=False)))